In [7]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/Data_Challenge/data.zip" -d /content/

Mounted at /content/drive


In [2]:
!rm -rf /content/Dropout_disco

In [3]:
!git clone https://github.com/EtiNL/Dropout_disco.git /content/Dropout_disco
# %cd content

import sys, os

repo_root = "/content/Dropout_disco"
if not os.path.isdir(repo_root):
    raise FileNotFoundError(f"Repo not found at {repo_root}")

if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

Cloning into '/content/Dropout_disco'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 0), reused 16 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 15.27 KiB | 7.64 MiB/s, done.


In [4]:
from load_data import load_train_data
from text_encoding import embed_texts
from training import train_clip_with_split
from eval_submission import build_submission_df

# Load data

In [8]:
motion_ids, motion_paths, map_df, text_df = load_train_data("./data")

# Encode text

In [9]:
text_encoder = "BAAI/bge-m3"

text_df, text_emb, _ = embed_texts(text_df, model_name=text_encoder)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Motion encoder

In [ ]:
from motion_encoder.lstm import LSTMMotionEncoder
from motion_encoder.simple_mlp import SimpleMotionEncoder

In [ ]:
x0 = np.load(motion_paths[0])
d_in = x0.shape[1]

motion_model = LSTMMotionEncoder(d_in=d_in, hidden_dim=256, out_dim=512)

# Training

In [ ]:
trained_clip, info = train_clip_with_split(
    motion_model=motion_model,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    epochs=50,
    batch_size=32,   # lower if VRAM tight
)

# Eval & build submission

In [ ]:
pred_df = build_submission_df(trained_clip, test_root="./test/test", top_k=10)
pred_df.head()

In [ ]:
pred_df.to_csv("./submission_bge-m3_LSTM2.csv", index=False)